In [1]:
import dsds.prescreen as ps
import dsds.transform as t
import dsds.sample as sa
import dsds.fs as fs 
import polars as pl

In [2]:
df = pl.scan_csv("../data/advertising.csv").sort(by="id")

In [3]:
df.limit(10).collect()

id,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad,One_Hot_Test,Age Band,Daily Internet Usage Band,Area Income Band,Test_Constant,Test_Str_Constant,Test_BadColumn,Test_Binary
i64,f64,i64,f64,f64,str,str,i64,str,str,i64,str,i64,i64,i64,i64,str,i64,str
1,68.95,35,61833.9,256.09,"""Cloned 5thgene…","""Wrightburgh""",0,"""Tunisia""","""3/27/2016 0:53…",0,"""A""",30,12,12,1,"""SSS""",0,"""A"""
2,80.23,31,68441.85,193.77,"""Monitored nati…","""West Jodi""",1,"""Nauru""","""4/4/2016 1:39""",0,"""B""",30,9,13,1,"""SSS""",null,"""B"""
3,69.47,26,59785.94,236.5,"""Organic bottom…","""Davidton""",0,"""San Marino""","""3/13/2016 20:3…",0,"""A""",20,11,11,1,"""SSS""",0,"""A"""
4,74.15,29,54806.18,245.89,"""Triple-buffere…","""West Terrifurt…",1,"""Italy""","""1/10/2016 2:31…",0,"""B""",20,12,10,1,"""SSS""",null,"""B"""
5,68.37,35,73889.99,225.58,"""Robust logisti…","""South Manuel""",0,"""Iceland""","""6/3/2016 3:36""",0,"""A""",30,11,14,1,"""SSS""",0,"""A"""
6,59.99,23,59761.56,226.74,"""Sharable clien…","""Jamieberg""",1,"""Norway""","""5/19/2016 14:3…",0,"""B""",20,11,11,1,"""SSS""",null,"""B"""
7,88.91,33,null,208.36,"""Enhanced dedic…","""Brandonstad""",0,"""Myanmar""","""1/28/2016 20:5…",0,"""A""",30,10,null,1,"""SSS""",0,"""A"""
8,66.0,48,24593.33,131.76,"""Reactive local…","""Port Jefferybu…",1,"""Australia""","""3/7/2016 1:40""",1,"""A""",40,6,4,1,"""SSS""",null,"""B"""
9,74.53,30,68862.0,221.51,"""Configurable c…","""West Colin""",1,"""Grenada""","""4/18/2016 9:33…",0,"""A""",30,11,13,1,"""SSS""",null,"""B"""


In [4]:
input_df = df.lazy()
output = input_df.pipe(ps.var_removal, threshold = 0.5, target = "Clicked on Ad")\
    .pipe(t.binary_encode)\
    .pipe(t.ordinal_auto_encode, cols = ["City", "Country"])\
    .pipe(t.impute, cols=["Daily Internet Usage", "Daily Internet Usage Band", "Area Income Band"], strategy="median")\
    .pipe(t.impute, cols=["Area Income"], strategy = "mean")\
    .pipe(t.scale, cols=["Area Income", "Daily Internet Usage"])\
    .pipe(t.one_hot_encode, cols= ["One_Hot_Test"])\
    .pipe(ps.remove_if_exists, cols = ["Ad Topic Line", "Timestamp"])\
    .pipe(fs.mutual_info_selector, target = "Clicked on Ad", top_k = 12)


INFO:dsds.prescreen:The following columns are dropped because they have lower than 0.5 variance. ['Male', 'Test_Constant', 'Test_BadColumn'].
Removed a total of 3 columns.
INFO:dsds.prescreen:The following columns are dropped. ['Ad Topic Line', 'Timestamp'].
Removed a total of 2 columns.
Mutual Info: 100%|██████████| 14/14 [00:00<00:00, 152.05it/s]

Selected 12 features. There are 2 columns the algorithm cannot process. They are also returned.


In [5]:
output.limit(100).collect()

Daily Internet Usage,Daily Internet Usage Band,Daily Time Spent on Site,Age,Age Band,Area Income,Area Income Band,One_Hot_Test_A,id,City,Country,Test_Binary_B,Clicked on Ad,Test_Str_Constant
f64,f64,f64,i64,i64,f64,f64,u8,i64,i64,i64,u8,i64,str
1.74127,12.0,68.95,35,30,0.511893,12.0,1,1,961,215,0,0,"""SSS"""
0.313948,9.0,80.23,31,30,1.005471,13.0,0,2,903,147,1,0,"""SSS"""
1.292598,11.0,69.47,26,20,0.358921,11.0,1,3,111,184,0,0,"""SSS"""
1.507658,12.0,74.15,29,20,-0.01304,10.0,0,4,939,103,1,0,"""SSS"""
1.042496,11.0,68.37,35,30,1.412418,14.0,1,5,805,96,0,0,"""SSS"""
1.069064,11.0,59.99,23,20,0.3571,11.0,0,6,282,158,1,0,"""SSS"""
0.648105,10.0,88.91,33,30,-1.0869e-15,11.0,1,7,46,145,0,0,"""SSS"""
-1.106273,6.0,66.0,48,40,-2.269777,4.0,1,8,671,12,1,1,"""SSS"""
0.94928,11.0,74.53,30,30,1.036854,13.0,1,9,884,82,1,0,"""SSS"""


In [6]:
output.blueprint

Step 0 | Action: drop
['Male', 'Test_Constant', 'Test_BadColumn']

Step 1 | Action: with_columns
Details: 
(1) .when([(col("Test_Binary")) == (Utf8(B))]).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("Test_Binary_B")


Step 2 | Action: drop
['Test_Binary']

Step 3 | Action: map_dict
MapDict(left_col='City', ref={'City': shape: (969,)
Series: '' [str]
[
	"Adamsbury"
	"Adamside"
	"Adamsstad"
	"Alanview"
	"Alexanderfurt"
	"Alexanderview"
	"Alexandrafort"
	"Alexisland"
	"Aliciatown"
	"Alvaradoport"
	"Alvarezland"
	"Amandafort"
	…
	"Williamsside"
	"Williamstad"
	"Wilsonburgh"
	"Wintersfort"
	"Wongland"
	"Wrightburgh"
	"Wrightview"
	"Yangside"
	"Youngburgh"
	"Youngfort"
	"Yuton"
	"Zacharystad"
	"Zacharyton"
], 'to': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67

In [7]:
output.blueprint.preserve("blueprint.pickle")

In [8]:
import pickle
f = open("blueprint.pickle", "rb")
pipeline = pickle.loads(f.read())
f.close()
# 
new_lf = pl.scan_csv("../data/advertising.csv").sort(by="id")

In [9]:
pipeline

Step 0 | Action: drop
['Male', 'Test_Constant', 'Test_BadColumn']

Step 1 | Action: with_columns
Details: 
(1) .when([(col("Test_Binary")) == (Utf8(B))]).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("Test_Binary_B")


Step 2 | Action: drop
['Test_Binary']

Step 3 | Action: map_dict
MapDict(left_col='City', ref={'City': shape: (969,)
Series: '' [str]
[
	"Adamsbury"
	"Adamside"
	"Adamsstad"
	"Alanview"
	"Alexanderfurt"
	"Alexanderview"
	"Alexandrafort"
	"Alexisland"
	"Aliciatown"
	"Alvaradoport"
	"Alvarezland"
	"Amandafort"
	…
	"Williamsside"
	"Williamstad"
	"Wilsonburgh"
	"Wintersfort"
	"Wongland"
	"Wrightburgh"
	"Wrightview"
	"Yangside"
	"Youngburgh"
	"Youngfort"
	"Yuton"
	"Zacharystad"
	"Zacharyton"
], 'to': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67

In [10]:
res = pipeline.apply(new_lf).collect()
res

Daily Internet Usage,Daily Internet Usage Band,Daily Time Spent on Site,Age,Age Band,Area Income,Area Income Band,One_Hot_Test_A,id,City,Country,Test_Binary_B,Clicked on Ad,Test_Str_Constant
f64,f64,f64,i64,i64,f64,f64,u8,i64,i64,i64,u8,i64,str
1.74127,12.0,68.95,35,30,0.511893,12.0,1,1,961,215,0,0,"""SSS"""
0.313948,9.0,80.23,31,30,1.005471,13.0,0,2,903,147,1,0,"""SSS"""
1.292598,11.0,69.47,26,20,0.358921,11.0,1,3,111,184,0,0,"""SSS"""
1.507658,12.0,74.15,29,20,-0.01304,10.0,0,4,939,103,1,0,"""SSS"""
1.042496,11.0,68.37,35,30,1.412418,14.0,1,5,805,96,0,0,"""SSS"""
1.069064,11.0,59.99,23,20,0.3571,11.0,0,6,282,158,1,0,"""SSS"""
0.648105,10.0,88.91,33,30,-1.0869e-15,11.0,1,7,46,145,0,0,"""SSS"""
-1.106273,6.0,66.0,48,40,-2.269777,4.0,1,8,671,12,1,1,"""SSS"""
0.94928,11.0,74.53,30,30,1.036854,13.0,1,9,884,82,1,0,"""SSS"""


In [11]:
# Perfect
(res.limit(100) == output.limit(100).collect()).sum()

Daily Internet Usage,Daily Internet Usage Band,Daily Time Spent on Site,Age,Age Band,Area Income,Area Income Band,One_Hot_Test_A,id,City,Country,Test_Binary_B,Clicked on Ad,Test_Str_Constant
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
100,100,100,100,100,100,100,100,100,100,100,100,100,100


# More Complicated Pipelines that are Beyond Sklearn

In [13]:
# This is a more advanced pipeline
# It contains filter, upsample steps, which are persisted
# which means they will be remembered by the pipeline and will be
# applied if the pipeline is applied to other dataframe.
# You can set persist = False so that these steps won't be remembered.

input_df = pl.scan_csv("../data/advertising.csv").sort(by="id")
output = input_df.pipe(ps.var_removal, threshold = 0.5, target = "Clicked on Ad")\
    .pipe(ps.filter, condition = pl.col("Age") > 30, persist = True)\
    .pipe(ps.constant_removal)\
    .pipe(sa.simple_upsample, subgroup = pl.col("One_Hot_Test") != 'A', count = 200, persist=True)\
    .pipe(t.binary_encode)\
    .pipe(t.one_hot_encode, cols= ["One_Hot_Test"])\
    .pipe(t.impute, cols=["Area Income", "Daily Internet Usage"], strategy="median")\
    .pipe(t.scale, cols=["Area Income", "Daily Internet Usage"])\
    .pipe(fs.mrmr_selector, target = "Clicked on Ad", top_k = 12, strategy="fscore")\
    .pipe(fs.mutual_info_selector, target = "Clicked on Ad", top_k=3)


INFO:dsds.prescreen:The following columns are dropped because they have lower than 0.5 variance. ['Male', 'Test_Constant', 'Test_BadColumn'].
Removed a total of 3 columns.
INFO:dsds.prescreen:The following columns are dropped because they are constants. ['Test_Str_Constant'].
Removed a total of 1 columns.


Running fscore to determine feature relevance...
Found 12 total features to select from. Proceeding to select top 12 features.


MRMR, fscore: 100%|██████████| 12/12 [00:00<00:00, 11998.01it/s]


Output is sorted in order of selection (max relevance min redundancy).
Selected 12 features. There are 5 columns the algorithm cannot process. They are also returned.


Mutual Info: 100%|██████████| 12/12 [00:00<00:00, 142.82it/s]

Selected 3 features. There are 5 columns the algorithm cannot process. They are also returned.


In [14]:
output.blueprint

Step 0 | Action: drop
['Male', 'Test_Constant', 'Test_BadColumn']

Step 1 | Action: filter
By condition: [(col("Age")) > (30)]


Step 2 | Action: drop
['Test_Str_Constant']

Step 3 | Action: apply_func
Function Module: dsds.sample, Function Name: simple_upsample
Parameters:
subgroup = [(col("One_Hot_Test")) != (Utf8(A))],
count = 200,
epsilon = 0.01,
include = None,
exclude = None,
positive = False,
seed = 42,


Step 4 | Action: with_columns
Details: 
(1) .when([(col("Test_Binary")) == (Utf8(B))]).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("Test_Binary_B")


Step 5 | Action: drop
['Test_Binary']

Step 6 | Action: with_columns
Details: 
(1) .when([(col("One_Hot_Test")) == (Utf8(A))]).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("One_Hot_Test_A")
(2) .when([(col("One_Hot_Test")) == (Utf8(B))]).then(1.strict_cast(UInt8)).otherwise(0.strict_cast(UInt8)).alias("One_Hot_Test_B")
(3) .when([(col("One_Hot_Test")) == (Utf8(C))]).then(1.strict_cast(UInt8

In [15]:
output.collect()

Daily Internet Usage,Daily Internet Usage Band,Daily Time Spent on Site,Ad Topic Line,City,Country,Timestamp,Clicked on Ad
f64,i64,f64,str,str,str,str,i64
1.8058,12,68.95,"""Cloned 5thgene…","""Wrightburgh""","""Tunisia""","""3/27/2016 0:53…",0
0.457549,9,80.23,"""Monitored nati…","""West Jodi""","""Nauru""","""4/4/2016 1:39""",0
1.145737,11,68.37,"""Robust logisti…","""South Manuel""","""Iceland""","""6/3/2016 3:36""",0
0.773194,10,88.91,"""Enhanced dedic…","""Brandonstad""","""Myanmar""","""1/28/2016 20:5…",0
-0.883995,6,66.0,"""Reactive local…","""Port Jefferybu…","""Australia""","""3/7/2016 1:40""",1
-1.094714,6,47.64,"""Centralized ne…","""West Brandonto…","""Qatar""","""3/16/2016 20:1…",1
1.260183,11,83.07,"""Team-oriented …","""East Theresash…","""Burundi""","""5/8/2016 8:10""",0
-1.287259,5,69.57,"""Centralized co…","""West Katiefurt…","""Egypt""","""6/3/2016 1:14""",1
-0.62871,7,42.95,"""Grass-roots co…","""West William""","""Barbados""","""3/24/2016 9:31…",1
